In [ ]:
import socket
import time
import threading
from pop import Gas, Dust, TempHumi, Pir, PiezoBuzzer, Touch, Led, PwmController, Textlcd

# ==========================================
# 1. 설정 및 초기화
# ==========================================
JAVA_IP = "192.168.101.102"  # PC IP 확인!
PORT_CMD    = 39186
PORT_SENSOR = 39187
PORT_DOOR   = 39189

# 객체 초기화
gas = Gas(6)
dust = Dust()
th = TempHumi()
pir = Pir(22)

PIN_BUZZER = 12
PIN_STATUS_LED = 27
lcd = Textlcd()
pwm = PwmController()
pwm.init()
pwm.setChannel(4)
touch = Touch()
p = PiezoBuzzer(PIN_BUZZER)
leds = Led(PIN_STATUS_LED)

# 변수
password = [10, 2, 10, 4]
input_pw = []
fail_count = 0
is_locked = True

# ==========================================
# 2. 도어락 함수
# ==========================================
def send_door_event(event):
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.settimeout(0.5)
        s.connect((JAVA_IP, PORT_DOOR))
        s.sendall((event + "\n").encode())
        s.close()
    except:
        pass

def play_tone(high):
    try:
        p.piezo.ChangeDutyCycle(0)
    except:
        pass
    try:
        if high:
            p.tone(5, 8, 10)
        else:
            p.tone(3, 1, 10)
    except:
        pass
    p.piezo.stop()

def lock_door():
    global is_locked
    print("\n[ACTION] 문 잠김")
    is_locked = True
    send_door_event("LOCKED")
    leds.off()
    pwm.setFreq(50)
    pwm.setDuty((1.6 / 20) * 100)
    lcd.clear()
    lcd.print("LOCKED")

def unlock_door():
    global is_locked
    print("\n[ACTION] 문 열림! (8초 후 자동 잠금)")
    is_locked = False
    send_door_event("UNLOCKED")
    leds.on()
    pwm.setFreq(50)
    pwm.setDuty((0.6 / 20) * 100)
    lcd.clear()
    lcd.print("OPEN")

    # 8초 후 자동 잠금
    threading.Timer(8.0, lock_door).start()

# ==========================================
# 3. 명령 수신 스레드 (패킷 분리 기능 포함)
# ==========================================
def command_listener_job():
    print(f"[Thread] 명령 서버({JAVA_IP}:{PORT_CMD}) 연결 대기중...")

    while True:
        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.connect((JAVA_IP, PORT_CMD))
            print("[System] Java 서버와 연결됨")

            while True:
                raw_data = sock.recv(1024).decode()

                if not raw_data:
                    print("[System] 서버 연결 끊김. 재접속 시도...")
                    break

                # 여러 명령이 한 번에 들어올 수 있으므로 개행 기준으로 분리
                commands = raw_data.split('\n')

                for cmd in commands:
                    cmd = cmd.strip().upper()

                    if not cmd:
                        continue

                    print(f"[처리 중] 명령: '{cmd}'")

                    if cmd == "UNLOCK":
                        unlock_door()
                    elif cmd == "LOCK":
                        lock_door()
                    elif cmd == "LED_ON":
                        leds.on()
                    elif cmd == "LED_OFF":
                        leds.off()
                    elif cmd == "REQ_FACE_UNLOCK":
                        print(" >> 얼굴 인식 요청 수신 (하드웨어 대기중)")

            sock.close()

        except:
            time.sleep(2)

t_cmd = threading.Thread(target=command_listener_job, daemon=True)
t_cmd.start()

# ==========================================
# 4. 센서 데이터 전송 스레드
# ==========================================
def sensor_thread_job():
    while True:
        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.connect((JAVA_IP, PORT_SENSOR))
            break
        except:
            time.sleep(2)

    while True:
        try:
            try:
                g_val = gas.readAverage()
                methan = gas.calcMethan(g_val)
                dust.read()
                pm1 = dust.pm_1p0_grimm
                pm25 = dust.pm_2p5_grimm
                pm10 = dust.pm_10_grimm
                temp = th.getTemperature()
                humi = th.getHumidity()
                pir_val = pir.read()
            except:
                pm1, pm25, pm10 = 0, 0, 0
                temp, humi = 0, 0
                pir_val = 0
                g_val, methan = 0, 0

            msg = (
                f"SENSOR GAS={g_val} METHAN={methan} "
                f"TEMP={temp:.2f} HUMI={humi:.2f} "
                f"PM1={pm1} PM25={pm25} PM10={pm10} PIR={pir_val}\n"
            )

            sock.sendall(msg.encode())
            time.sleep(1)

        except:
            break

t_sensor = threading.Thread(target=sensor_thread_job, daemon=True)
t_sensor.start()

# ==========================================
# 5. 메인 루프: 터치 키패드 비밀번호 처리
# ==========================================
print("\n[Main] 시스템 정상 가동 중...")
lock_door()

try:
    while True:
        for ch in range(12):
            if touch.readChannel(ch) == 1:

                key_char = '*' if ch == 9 else ('#' if ch == 11 else str(ch))
                print(f"키 입력: {key_char}")

                p.tone(4, 4, 5)
                p.piezo.stop()

                input_pw.append(ch)

                if len(input_pw) == len(password):

                    if input_pw == password:
                        play_tone(True)
                        unlock_door()
                    else:
                        play_tone(False)
                        fail_count += 1

                        if fail_count >= 3:
                            send_door_event("ALERT_FAIL_3")
                            time.sleep(3)
                            fail_count = 0

                    input_pw = []

                time.sleep(0.3)

        time.sleep(0.05)

except KeyboardInterrupt:
    print("종료")
